### Učitavanje i ispis jelovnika. 

In [60]:
from parsers import *
from formatting import *
import time

model = LpProblem("Keto_Diet_Problem", LpMinimize)

parser = create_parser("jelovnik.xls")
print(parser)

                              Jela  Cijena u HRK  Kalorije  Masti [g]  \
0                 Smrznuta brokula      1.121136      73.8        0.8   
1                            Mrkva      0.490497      23.7        0.1   
2                            Celer      0.280284       6.4        0.1   
3                  Smrznut kukuruz      1.261278      72.2        0.6   
4             Svježa zelena salata      0.140142       2.6        0.0   
5                   Slatke paprike      3.713763      20.0        0.1   
6                  Pečeni krumpiri      0.420426     171.5        0.2   
7                             Tofu      2.172201      88.2        5.5   
8                  Pečena piletina      5.885964     277.4       10.8   
9                 Špagete s umakom      5.465538     358.2       12.3   
10                         Rajčica      1.891917      25.8        0.4   
11                  Jabuka s korom      1.681704      81.4        0.5   
12                          Banana      1.051065   

### Učitavanje stupaca tablice i spremanje u varijable. 

In [53]:
food_list = parser_to_list(parser, "Jela")
costs = parser_to_dict(parser, food_list, 'Cijena u HRK')
calories = parser_to_dict(parser, food_list, 'Kalorije')
fats = parser_to_dict(parser, food_list, 'Masti [g]')
carbs = parser_to_dict(parser, food_list, 'Ugljikohidrati [g]')
fibers = parser_to_dict(parser, food_list, 'Vlakna [g]')
proteins = parser_to_dict(parser, food_list, 'Proteini [g]')

In [54]:
variables = LpVariable.dicts("Food", food_list, 0, cat='Integer')
model += lpSum([costs[i]*variables[i] for i in food_list]), "Total cost of the keto diet"

## Korisničko dodavanje ograničenja.

In [55]:
# Potrebno mijenjati vrijednosti intervala unutar kojega želimo jelovnik s [g] masnoće.
model += lpSum([fats[i] * variables[i] for i in food_list]) >= 100.0, "Fat Minimum"
model += lpSum([fats[i] * variables[i] for i in food_list]) <= 220.0, "Fat Maximum"

# Potrebno mijenjati vrijednosti intervala unutar kojega želimo jelovnik s [g] ugljikohidrata.
model += lpSum([carbs[i] * variables[i] for i in food_list]) >= 10.0, "Carbs Minimum"
model += lpSum([carbs[i] * variables[i] for i in food_list]) <= 20.0, "Carbs Maximum"

# Potrebno mijenjati vrijednosti intervala unutar kojega želimo jelovnik s [g] vlakana.
model += lpSum([fibers[i] * variables[i] for i in food_list]) >= 5.0, "Fiber Minimum"
model += lpSum([fibers[i] * variables[i] for i in food_list]) <= 10.0, "Fiber Maximum"

# Potrebno mijenjati vrijednosti intervala unutar kojega želimo jelovnik s [g] proteina.
model += lpSum([proteins[i] * variables[i] for i in food_list]) >= 150.0, "Protein Minimum"
model += lpSum([proteins[i] * variables[i] for i in food_list]) <= 250.0, "Protein Maximum"

# Potrebno mijenjati vrijednosti intervala unutar kojega želimo jelovnik s kalorijama.
model += lpSum([calories[i] * variables[i] for i in food_list]) >= 1800.0, "Calorie Minimum"
model += lpSum([calories[i] * variables[i] for i in food_list]) <= 2500.0, "Calorie Maximum"

### Ograničenje porcija istog jela.

In [56]:
# Potrebno mijenjati vrijednosti do koje korisnik želi najveći broj jednakih porcija.
for food in food_list:
    model += lpSum(variables[food]) <= 3, str(food) + " maximum."

### Rješavanje problema i ispis vremena potrebnog za rješavanje.

In [62]:
model.writeLP('KetoDietProblemJupyter.lp')
start_time = time.time()
model.solve()
print("\nTime: ", str(round(time.time() - start_time, 5)), "s")


Time:  0.01823 s


### Ispis rješenja problema.

In [58]:
formatted_print(model)


Status: Optimal
Items to buy -> Amount:

Kajgana -> 2.0
Maslac_od_kikirikija -> 2.0
Pečena_piletina -> 2.0
Poširana_jaja -> 3.0
Purica,_Bologna -> 1.0
Svinjetina -> 1.0
Svježa_zelena_salata -> 3.0
Total cost: 23.12 HRK


### Ispis postupka rješavanja.

In [59]:
with open('KetoDietProblemJupyter.lp') as file:
    for line in file:
        print(line.strip())

\* Keto_Diet_Problem *\
Minimize
Total_cost_of_the_keto_diet: 1.051065 Food_Banana + 0.560568 Food_Bijela_riža
+ 4.834899 Food_Bijela_tuna_u_vodi + 0.420426 Food_Bijeli_kruh
+ 0.280284 Food_Celer + 1.751775 Food_Cheddar_sir
+ 1.891917 Food_Govedina,_Frankfurter + 4.694757 Food_Grah
+ 2.242272 Food_Grejp + 5.815893 Food_Hamburger + 2.172201 Food_Hotdog
+ 1.681704 Food_Jabuka_s_korom + 4.554615 Food_Juha_od_gljiva
+ 4.694757 Food_Juha_od_graška + 4.975041 Food_Juha_od_povrća
+ 2.732769 Food_Juha_od_rajčice + 5.255325 Food_Juha_s_plodovima_mora
+ 0.770781 Food_Kajgana + 0.210213 Food_Keksi_s_čokoladnim_komadima
+ 3.433479 Food_Kivi + 1.051065 Food_Kobasica,_Prk + 0.280284 Food_Kokice
+ 0.630639 Food_Kolačići_od_zobene_kašice + 2.732769 Food_Kuskus
+ 1.191207 Food_Makaroni + 0.350355 Food_Maslac,_običan
+ 0.490497 Food_Maslac_od_kikirikija + 1.121136 Food_Mlijeko_s_3,3%_masti
+ 1.611633 Food_Mlijeko_s_malo_masnoće + 0.490497 Food_Mrkva
+ 1.051065 Food_Narače + 0.910923 Food_Obrano_mlijeko
